In [ ]:
import math
import numpy as np
from typing import Tuple

# Araç Dinamiği ve MPPI Kontrollü Sistem
class MPPIControllerForVehicle():
    def __init__(
            self,
            delta_t: float = 0.05,  # Zaman adımı [s]
            mass: float = 100.0,  # Araç kütlesi [kg]
            inertia_matrix: np.ndarray = np.eye(3) * 10.0,  # Atalet matrisi
            damping_matrix: np.ndarray = np.eye(3) * 5.0,  # Sönümleme matrisi
            propeller_forces: np.ndarray = np.zeros(6),  # Pervane kuvvetleri
            ref_path: np.ndarray = np.array([[-100.0, 0.0], [100.0, 0.0]]),
            horizon_step_T: int = 30,  # MPPI tahmin penceresi
            number_of_samples_K: int = 1000,  # Örnekleme sayısı
            param_lambda: float = 50.0,  # MPPI lambda parametresi
            sigma: np.ndarray = np.array([[0.5, 0.0], [0.0, 0.1]]),  
            visualize: bool = True  # Görselleştirme bayrağı
    ) -> None:
        """
        MPPI algoritmasını kullanarak araç kontrolü için bir sınıf.
        """
        self.delta_t = delta_t  # Zaman adımı
        self.mass = mass  # Araç kütlesi
        self.M = inertia_matrix  # Atalet matrisi
        self.D = damping_matrix  # Sönümleme matrisi
        self.propeller_forces = propeller_forces  # Pervane kuvvetleri
        self.ref_path = ref_path  # Referans yol
        self.T = horizon_step_T  # Tahmin penceresi
        self.K = number_of_samples_K  # MPPI örnekleme sayısı
        self.param_lambda = param_lambda  # MPPI lambda parametresi
        self.Sigma = sigma  # Gürültü sapması
        self.visualize = visualize  # Görselleştirme ayarı
        
        # Araç durumu [x, y, z, roll, pitch, yaw, u, v, w, p, q, r]
        self.state = np.zeros(12)
        
        # Önceki kontrol girdisi
        self.u_prev = np.zeros((self.T, 6))

    def calc_control_input(self, observed_x: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Optimal kontrol girdisini hesaplar.
        """
        u = self.u_prev  # Önceki kontrol girdisini yükle
        x0 = observed_x  # Başlangıç durumu
        S = np.zeros((self.K))  # Durum maliyet listesi
        epsilon = self._calc_epsilon(self.Sigma, self.K, self.T, 6)  # Gürültü örnekleme
        
        for k in range(self.K):  # MPPI örnekleme döngüsü
            v = np.zeros((self.T, 6))  # Kontrol girdisi dizisi
            x = x0  # Başlangıç durumu
            
            for t in range(self.T):
                v[k, t] = u[t] + epsilon[k, t]  # Rastgele keşif
                x = self._F(x, self._g(v[k, t]))  # Yeni durumu hesapla
                S[k] += self._cost_function(x)  # Maliyet fonksiyonunu hesapla

            S[k] += self._terminal_cost(x)  # Terminal maliyet ekle
        
        w = self._compute_weights(S)  # MPPI ağırlıkları hesapla
        w_epsilon = np.zeros((self.T, 6))
        
        for t in range(self.T):
            for k in range(self.K):
                w_epsilon[t] += w[k] * epsilon[k, t]
        
        u += w_epsilon  # Kontrol girdisini güncelle
        self.u_prev[:-1] = u[1:]
        self.u_prev[-1] = u[-1]
        
        return u[0], u  # Optimal kontrol girdisini döndür
    
    def _F(self, x: np.ndarray, u: np.ndarray) -> np.ndarray:
        """
        Araç dinamiği güncellemesi (6 DOF).
        """
        M_inv = np.linalg.inv(self.M)  # Atalet matrisi tersini al
        v = x[6:12]  # Lineer ve açısal hız vektörü
        tau = self._compute_tau(u)  # Kuvvet ve moment vektörü
        
        # Lineer ve açısal ivmeleri hesapla
        v_dot = np.matmul(M_inv, tau - np.matmul(self.D, v))
        
        # Yeni durumu hesapla
        x_new = x.copy()
        x_new[0:6] += v * self.delta_t  # Pozisyon ve açı güncellemesi
        x_new[6:12] += v_dot * self.delta_t  # Hız güncellemesi
        
        return x_new

    def _compute_tau(self, u: np.ndarray) -> np.ndarray:
        """
        Kuvvet ve moment vektörünü hesaplar.
        """
        F = np.zeros(6)  # [Fx, Fy, Fz, Mx, My, Mz]
        F[0:3] = np.sum(self.propeller_forces[:, None] * u, axis=0)  # Lineer kuvvetler
        F[3:6] = np.cross(self.propeller_forces[:, None], u).sum(axis=0)  # Momentler
        return F

    def _cost_function(self, x: np.ndarray) -> float:
        """
        Araç için aşama maliyet fonksiyonu.
        """
        error = np.linalg.norm(x[0:3] - self.ref_path[0, 0:3])  # Pozisyon hatası
        return error ** 2
    
    def _terminal_cost(self, x: np.ndarray) -> float:
        """
        Araç için terminal maliyet fonksiyonu.
        """
        error = np.linalg.norm(x[0:3] - self.ref_path[-1, 0:3])  # Hedef nokta hatası
        return error ** 2

    def _compute_weights(self, S: np.ndarray) -> np.ndarray:
        """
        MPPI ağırlıklarını hesaplar.
        """
        w = np.exp(-S / self.param_lambda)
        return w / np.sum(w)

    def _calc_epsilon(self, sigma: np.ndarray, size_sample: int, size_time_step: int, size_dim_u: int) -> np.ndarray:
        """
        Gürültü örnekleme işlemi.
        """
        mu = np.zeros((size_dim_u))
        return np.random.multivariate_normal(mu, sigma, (size_sample, size_time_step))

# MPPI tabanlı kontrolcü ile simülasyonu başlat
if __name__ == "__main__":
    print("[INFO] MPPI ile araç simülasyonu başlatılıyor...")
    vehicle = MPPIControllerForVehicle()
    state = np.zeros(12)
    
    for _ in range(200):
        u_opt, _ = vehicle.calc_control_input(state)
        state = vehicle._F(state, u_opt)  # Araç durumu güncelle
        print(f"Durum: {state}")
